In [ ]:
### TOPOGRAPHY ###

In [1]:
import geopandas as gpd
import georasters as grs
import pandas as pd
from shapely.geometry import Point, Polygon

In [2]:
# set coords in the proper metric crs and radius in meters
# coords = (-40113.87, 164213.15) # Porto, metris crs - EPSG:3763
# coords = (-87519, -105502) # Lisbon, metris crs - EPSG:3763
# coords = (51726.18613283, 28206.23996747) # San Francisco, metris crs - EPSG:7131
coords = (835296.86, 820837.43) # Hong Kong, metris crs - EPSG:2326

center = gpd.GeoDataFrame(geometry = [Point(coords)],
                          crs = 'epsg:2326')
radius = 2000

# metric dimensions for the topography grid
max_x = coords[0] + radius
max_y = coords[1] + radius
min_x = coords[0] - radius
min_y = coords[1] - radius

mask = gpd.GeoDataFrame(geometry = [Polygon([(max_x, max_y), (max_x, min_y), (min_x, min_y), (min_x, max_y)])],
                        crs = 'epsg:2326')

In [5]:
# set path variables
folder = r'C:/folder'
file = '/hong_kong_clipped_m.tif'
path = folder + file
output = folder + '/HK_topo.csv'

C:\Eli\Arch\PhD\Papers\3\topo\hong_kong_clipped_m.tif


In [6]:
# load elevation data
data = grs.from_file(path)

In [7]:
# turn to pandas dataframe
df = data.to_pandas()
df.head(10)

,row,col,value,x,y
0,0,1,3.0,818213.738911,833961.58007
1,0,2,3.0,818242.930281,833961.58007
2,0,3,2.0,818272.121650,833961.58007
3,0,4,3.0,818301.313019,833961.58007
4,0,5,8.0,818330.504389,833961.58007
5,0,6,8.0,818359.695758,833961.58007
6,0,7,11.0,818388.887127,833961.58007
7,0,8,11.0,818418.078497,833961.58007
8,0,9,10.0,818447.269866,833961.58007
9,0,10,6.0,818476.461235,833961.58007


In [8]:
# create point geometry and set crs
gdf = gpd.GeoDataFrame(df,
                       geometry = gpd.points_from_xy(df.x, df.y),
                       crs = 2326)
print(gdf.head(10))
print(gdf.shape)

   row  col  value              x             y                       geometry
0    0    1    3.0  818213.738911  833961.58007  POINT (818213.739 833961.580)
1    0    2    3.0  818242.930281  833961.58007  POINT (818242.930 833961.580)
2    0    3    2.0  818272.121650  833961.58007  POINT (818272.122 833961.580)
3    0    4    3.0  818301.313019  833961.58007  POINT (818301.313 833961.580)
4    0    5    8.0  818330.504389  833961.58007  POINT (818330.504 833961.580)
5    0    6    8.0  818359.695758  833961.58007  POINT (818359.696 833961.580)
6    0    7   11.0  818388.887127  833961.58007  POINT (818388.887 833961.580)
7    0    8   11.0  818418.078497  833961.58007  POINT (818418.078 833961.580)
8    0    9   10.0  818447.269866  833961.58007  POINT (818447.270 833961.580)
9    0   10    6.0  818476.461235  833961.58007  POINT (818476.461 833961.580)
(1557026, 6)


In [9]:
# intersect the topography with the bounding box
area = gdf.sjoin(mask, how = 'left', predicate = 'intersects')
area = area[area.index_right.notnull()]
area.drop(columns = 'index_right', inplace = True)
print(area.shape)
area.head()

(18769, 6)


,row,col,value,x,y,geometry
596398,382,518,109.0,833305.676863,822810.47698,POINT (833305.677 822810.477)
596399,382,519,116.0,833334.868232,822810.47698,POINT (833334.868 822810.477)
596400,382,520,118.0,833364.059602,822810.47698,POINT (833364.060 822810.477)
596401,382,521,119.0,833393.250971,822810.47698,POINT (833393.251 822810.477)
596402,382,522,128.0,833422.442340,822810.47698,POINT (833422.442 822810.477)


In [10]:
# find grid dimensions
u = (area['row'].max()) - (area['row'].min())
v = (area['col'].max()) - (area['col'].min())
print(u)
print(v)
# Value u+1 should be used as U in SrfGrid GH component.

136
136


In [11]:
point_series = area.iloc[:,-1].tolist()
x_vals = []
y_vals = []
z_vals = []
row = area.iloc[:, 0].tolist()
col = area.iloc[:, 1].tolist()
for i, pt in enumerate(point_series):
    x_vals.append(pt.coords[0][0])
    y_vals.append(pt.coords[0][1])
    z_vals.append(area.iloc[i, 2])

In [12]:
data_dict = {'x': x_vals, 'y': y_vals, 'z': z_vals, 'col': col, 'row': row}
point_map = pd.DataFrame(data = data_dict)
print(point_map.shape)
point_map.head()

(18769, 5)


,x,y,z,col,row
0,833305.676863,822810.47698,109.0,518,382
1,833334.868232,822810.47698,116.0,519,382
2,833364.059602,822810.47698,118.0,520,382
3,833393.250971,822810.47698,119.0,521,382
4,833422.442340,822810.47698,128.0,522,382


In [13]:
# export csv
point_map.to_csv(path_or_buf = output, header = False, index = False, encoding = 'utf8', line_terminator = '')

In [14]:
# add grid dimensions to the csv file
file = open(output, 'a')
file.write(str(u+1))
file.write('\n')
file.close()